In [1]:
abstract type AbstractPoint end

struct PointI <: AbstractPoint
    x::Int
    y::Int
end
   
mutable struct PointM <: AbstractPoint
    x::Int
    y::Int
end

PointM(p::PointM) = PointM(p.x, p.y)

PointM

In [2]:
d(p::AbstractPoint) = abs(p.x) + abs(p.y)
move(p::PointI, d::PointI) = PointI(p.x+d.x, p.y+d.y)
move(p::PointM, d::PointM) = (p.x += d.x; p.y += d.y; p)

move (generic function with 2 methods)

In [3]:
function simI()
    maxd = 0
    x = PointI(0, 0)
    @inbounds for i in 1:10^6
        x = move(x, PointI(2rand(Bool)-1, 2rand(Bool)-1))
        curd = d(x)
        maxd = max(maxd, curd)
    end
    maxd
end

function simM()
    maxd = 0
    x = PointM(0, 0)
    m = PointM(0, 0)
    @inbounds for i in 1:10^6
        m.x, m.y = 2rand(Bool)-1, 2rand(Bool)-1
        move(x, m)
        curd = d(x)
        maxd = max(maxd, curd)
    end
    maxd
end

simM (generic function with 1 method)

In [4]:
using BenchmarkTools

In [5]:
@benchmark simI()

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.034 ms (0.00% GC)
  median time:      3.181 ms (0.00% GC)
  mean time:        3.285 ms (0.00% GC)
  maximum time:     6.015 ms (0.00% GC)
  --------------
  samples:          1520
  evals/sample:     1

In [6]:
@benchmark simM()

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.034 ms (0.00% GC)
  median time:      3.213 ms (0.00% GC)
  mean time:        3.362 ms (0.00% GC)
  maximum time:     4.556 ms (0.00% GC)
  --------------
  samples:          1485
  evals/sample:     1

In [7]:
function simI2()
    path = PointI[]
    x = PointI(0, 0)
    @inbounds for i in 1:10^6
        push!(path, x)
        x = move(x, PointI(2rand(Bool)-1, 2rand(Bool)-1))
    end
    path
end

function simM2()
    path = PointM[]
    x = PointM(0, 0)
    m = PointM(0, 0)
    @inbounds for i in 1:10^6
        push!(path, PointM(x))
        m.x, m.y = 2rand(Bool)-1, 2rand(Bool)-1
        move(x, m)
    end
    path
end

simM2 (generic function with 1 method)

In [8]:
@benchmark simI2()

BenchmarkTools.Trial: 
  memory estimate:  17.00 MiB
  allocs estimate:  20
  --------------
  minimum time:     11.275 ms (0.00% GC)
  median time:      15.660 ms (0.00% GC)
  mean time:        15.891 ms (8.13% GC)
  maximum time:     25.279 ms (33.32% GC)
  --------------
  samples:          294
  evals/sample:     1

In [9]:
@benchmark simM2()

BenchmarkTools.Trial: 
  memory estimate:  39.52 MiB
  allocs estimate:  1000020
  --------------
  minimum time:     19.824 ms (0.00% GC)
  median time:      37.589 ms (49.03% GC)
  mean time:        39.181 ms (51.67% GC)
  maximum time:     121.914 ms (82.95% GC)
  --------------
  samples:          128
  evals/sample:     1

### ーーーーーーーー

In [11]:
struct T1
    x::NTuple{1000, Int}
    y::Int
end

mutable struct T2
    x::NTuple{1000, Int}
    y::Int
end

function worker1()
    p = T1(ntuple(x->1, 1000), 0)
    for i in 1:10^6
        p = T1(p.x, p.y+1)
    end
    p
end

function worker2()
    p = T2(ntuple(x->1, 1000), 0)
    for i in 1:10^6
        p.y += 1
    end
    p 
end

worker2 (generic function with 1 method)

In [12]:
@benchmark worker1()

BenchmarkTools.Trial: 
  memory estimate:  31.75 KiB
  allocs estimate:  6
  --------------
  minimum time:     268.387 μs (0.00% GC)
  median time:      276.055 μs (0.00% GC)
  mean time:        296.886 μs (1.93% GC)
  maximum time:     38.758 ms (99.19% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [13]:
@benchmark worker2()

BenchmarkTools.Trial: 
  memory estimate:  39.69 KiB
  allocs estimate:  7
  --------------
  minimum time:     24.520 μs (0.00% GC)
  median time:      30.552 μs (0.00% GC)
  mean time:        38.356 μs (17.06% GC)
  maximum time:     47.323 ms (99.86% GC)
  --------------
  samples:          10000
  evals/sample:     1